These examples show how to create GW-BSE workflow using $py$GWBSE
package. We will show how to compute QP bandstructure for wurtzite-AlN
at the partial self-consistent (GW$_0$) level by obtaining input 
structure from Material Project (MP) database. We will also compute the 
absorption spectra of wurtzite-AlN by solving Bethe-Salpeter equation (BSE).

In [1]:
#First let's load the required libraries such as pymatgen and Fireworks
import sys
from pyGWBSE.make_wflow import read_input, create_wfs
from fireworks import LaunchPad
from pymatgen.ext.matproj import MPRester
from pymatgen import Structure

In [9]:
#Now let's check the content of the 'input.yaml' file.
!cat 'input.yaml'

PARAMS:
  kpar: 8                   
  # 'KPAR' tag to be used in VASP simulations
  
  ppn: 13                   
  # NPROCS/KPAR ; NPROCS: number of total processors to be used in VASP simulations  
  
  reciprocal_density: 200   
  # reciprocal density that determines the k-grid using 'automatic_density_by_vol' method of pymatgen
  
  encutgw: 100              
  # ENCUTGW to be used in VASP simulations
  
  nbgwfactor: 3             
  # NBANDS=nbgwfactor x nocc'; nocc' is the smallest multiple of ppn which is larger than number of occupied bands
  
  nomegagw: 80              
  # NOMEGAGW to be used in VASP simulations
  
  convparam: NBANDS         
  # NBANDS/ENCUTGW/NOMEGA parameter to run convergence test for  
  
  convsteps: 2              
  # steps to be used for convergence test
  
  conviter: 5               
  # maximum number of iteration in convergence test 
  
  enwinbse: 3.0             
  # energy window in BSE calculations



STRUCTURE:
  
  source: MID          

In [3]:
"""
    'read_input' function reads input parameters from input.yaml file.
    
     Returns:
    
     structure: pymatgen structure object for the input structure
     input_dict: a dictionary of all the input paramters
     
"""

structure, input_dict = read_input()

In [4]:
"""
    'create_wfs' function creates the GW-BSE workflow based on input specification.
    
    Args:
    
     structure: pymatgen structure object for the input structure
     input_dict: a dictionary of all the input paramters
     
    Returns:
    
     wf_gwbse: GW-BSE workflow as a Fireworks workflow object.
         
"""

wf_gwbse=create_wfs(structure, input_dict)

-------------------------------------------
material:  mp-661
Information for efficient parallelization
You have  8 occupied bands
You have  76 kpoints
You have  [[12, 12, 7]] k-grid
Will perform convergence test for:  NBANDS
Values that will be used for convergence test: 39 65 91 117 143 
KPAR= 8
reciprocal_density= 200
BSE calculation will include bands in the energy window (eV)= 3.0
-------------------------------------------


/home/tatha/anaconda3/envs/cms/lib/python3.6/site-packages/pymatgen/io/vasp/sets.py:447: BadInputSetWarning:

Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.



The error message "Overriding the POTCAR functional is generally not ..." is due to changing default POTCAR symbols to GW POTCAR files. One can choose other POTCARs by modifying inputset.yaml file. 

In [5]:
#Load the Fireworks Launchpad 
lpad = LaunchPad.auto_load() 

In [6]:
#Add the workflow you just created
lpad.add_wf(wf_gwbse)

2022-09-14 14:27:54,737 INFO Added a workflow. id_map: {-7: 1, -6: 2, -5: 3, -4: 4, -3: 5, -2: 6, -1: 7}


{-7: 1, -6: 2, -5: 3, -4: 4, -3: 5, -2: 6, -1: 7}

In [7]:
#Check the Fireworks loaded to the Launchpad
lp='lpad get_fws'
!{lp}

/home/tatha/anaconda3/envs/cms/lib/python3.6/site-packages/pymatgen/io/vasp/sets.py:447: BadInputSetWarning:

Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.

[
    {
        "fw_id": 1,
        "created_on": "2022-09-14T21:27:54.144377",
        "updated_on": "2022-09-14T21:27:54.144378",
        "state": "PAUSED",
        "name": "mp-661-BSE"
    },
    {
        "fw_id": 2,
        "created_on": "2022-09-14T21:27:54.144357",
        "updated_on": "2022-09-14T21:27:54.144358",
        "name": "mp-661-WANNIER",
        "state": "WAITING"
    },
    {
        "fw_id": 3,
        "created_on": "2022-09-14T21:27:54.144341",
        "updated_on": "2022-09-14T21:27:54.144341",
        "name": "mp-661-GW",
        "state": "WAITING"
    }

As you can see there are 7 fireworks with prefix `mp-661` were loaded to the Launchpad. 
Also one should note that the BSE Firework is in a `PAUSED` state by default. This is 
because, the k-point parallelization is not implemented for BSE calculation in VASP. 
Therefore, to make the number of bands in the WAVEDER file a multiple of number of 
processor used (requirement for VASP, otherwise it will change the NBANDS value) we 
need to run the BSE workflow on NPROCS=NPROCS$'$/KPAR, where NPROCS$'$ is the number 
of processors used for the `fw_id: 2-7`. Now that we have all the Fireworks in the 
Launchpad we can run them and use the postprocessing scripts to analyze the results once 
they are `COMPLETED`.